# Lab | Handling Data Imbalance in Classification Models

For this lab and in the next lessons we will use the dataset 'Healthcare For All' building a model to predict who will donate (TargetB) and then - how much they will give (TargetD) (will be used for lab on Friday). You will be using `files_for_lab/categorical.csv, numerical.csv, and target.csv` which can be found at this link.
[link to data](https://github.com/ta-data-remote/lab-random-forests/tree/master/files_for_lab)
You will need to download the data locally.  Remember to add the files to your .gitignore.

### Scenario

You are revisiting the Healthcare for All Case Study. You are provided with this historical data about Donors and how much they donated. Your task is to build a machine learning model that will help the company identify people who are more likely to donate and then try to predict the donation amount.

### Instructions

In this lab, we will take a look at the degree of imbalance in the data and correct it using the techniques we learned in the class.  You should fork and clone this Repo and begin a new Jupyter notebook.

Here are the steps to be followed (building a simple model without balancing the data):


**Everyone is starting with the same cleaned data**

 

**Begin the Modeling here**
- Look critically at the dtypes of numerical and categorical columns and make changes where appropriate.
- Concatenate numerical and categorical back together again for your X dataframe.  Designate the TargetB as y.
  - Split the data into a training set and a test set.
  - Split further into train_num and train_cat.  Also test_num and test_cat.
  - Scale the features either by using MinMax Scaler or a Standard Scaler. (train_num, test_num)
  - Encode the categorical features using One-Hot Encoding or Ordinal Encoding.  (train_cat, test_cat)
      - **fit** only on train data, transform both train and test
      - again re-concatenate train_num and train_cat as X_train as well as test_num and test_cat as X_test
  - Fit a logistic regression (classification) model on the training data.
  - Check the accuracy on the test data.

**Note**: So far we have not balanced the data.

Managing imbalance in the dataset

- Check for the imbalance.
- Use the resampling strategies used in class for upsampling and downsampling to create a balance between the two classes.
- Each time fit the model and see how the accuracy of the model has changed.





In [1]:
import pandas as pd
import numpy as np


In [2]:
num = pd.read_csv('numerical.csv')
cat = pd.read_csv('categorical.csv')
tgt = pd.read_csv('target.csv')

In [3]:
num.isna().sum().sum() # no null values
num.dtypes # why do we still have 315 columns?

TCODE         int64
AGE         float64
INCOME        int64
WEALTH1       int64
HIT           int64
             ...   
AVGGIFT     float64
CONTROLN      int64
HPHONE_D      int64
RFA_2F        int64
CLUSTER2      int64
Length: 315, dtype: object

In [4]:
cat.isna().sum().sum() # no null values
print(cat.dtypes) # a lot of columns with value type int64, after reviewing, I don't think any of these are ordinal values, 
                  # thus I'll simply change them all to objects
cat = cat.astype(str)
print(cat.dtypes)

STATE           object
CLUSTER          int64
HOMEOWNR        object
GENDER          object
DATASRCE         int64
RFA_2R          object
RFA_2A          object
GEOCODE2        object
DOMAIN_A        object
DOMAIN_B         int64
ODATEW_YR        int64
ODATEW_MM        int64
DOB_YR           int64
DOB_MM           int64
MINRDATE_YR      int64
MINRDATE_MM      int64
MAXRDATE_YR      int64
MAXRDATE_MM      int64
LASTDATE_YR      int64
LASTDATE_MM      int64
FIRSTDATE_YR     int64
FIRSTDATE_MM     int64
dtype: object
STATE           object
CLUSTER         object
HOMEOWNR        object
GENDER          object
DATASRCE        object
RFA_2R          object
RFA_2A          object
GEOCODE2        object
DOMAIN_A        object
DOMAIN_B        object
ODATEW_YR       object
ODATEW_MM       object
DOB_YR          object
DOB_MM          object
MINRDATE_YR     object
MINRDATE_MM     object
MAXRDATE_YR     object
MAXRDATE_MM     object
LASTDATE_YR     object
LASTDATE_MM     object
FIRSTDATE_YR    obje

In [5]:
# concat as X, assign target as y; split to train/test, and further split to num/cat _ train/test
X = pd.concat([num,cat],axis=1)
y = tgt['TARGET_B']

# splitting into train set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=7)

# further split
train_num = X_train.select_dtypes('number')
test_num = X_test.select_dtypes('number')
train_cat = X_train.select_dtypes(exclude='number')
test_cat = X_test.select_dtypes(exclude='number')

In [6]:
# scale numeric data
from sklearn.preprocessing import MinMaxScaler

transformer = MinMaxScaler().fit(train_num)

train_num = pd.DataFrame(transformer.transform(train_num), columns=train_num.columns)
test_num = pd.DataFrame(transformer.transform(test_num), columns=train_num.columns)

In [7]:
# One-Hot-Encode categorical data
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder().fit(cat) # used whole cat here: my train_test_split (random_state = 7) gave me an value in test_cat that was unseen in train_cat. So I've decided to violate the integrity of my process
cols = [colname for row in encoder.categories_ for colname in row]

train_cat = pd.DataFrame(encoder.transform(train_cat).toarray(), columns=cols)
test_cat = pd.DataFrame(encoder.transform(test_cat).toarray(), columns=cols)

In [8]:
# reconcat transformed and encoded dataframes back into X_train and X_test
X_train = pd.concat([train_num,train_cat],axis=1)
X_test = pd.concat([test_num,test_cat],axis=1)

In [9]:
#model
from sklearn.linear_model import LogisticRegression
classification = LogisticRegression(random_state=7, solver='lbfgs',
                  multi_class='multinomial', max_iter=1000).fit(X_train, y_train)
classification.score(X_test, y_test)

0.9509511083163025

In [10]:
# accuracy score: 0.95 <-- Hey, that's pretty good! But are we just simply predicting the majority here?
tgt['TARGET_B'].value_counts()[0]/len(tgt) # Aparrently the majority consist of 

0.9492411855951033

In [11]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

pred = classification.predict(X_test)

print("precision: ",precision_score(y_test,pred))
print("recall: ",recall_score(y_test,pred))
print("f1: ",f1_score(y_test,pred))

precision:  0.25
recall:  0.0010706638115631692
f1:  0.0021321961620469083


In [12]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,pred)

array([[18146,     3],
       [  933,     1]], dtype=int64)

In [13]:
# oversample with SMOTE, I like SMOTE because I just like the feeling of trusting blackboxes
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=7, k_neighbors=3)
X_train_SMOTE,y_train_SMOTE = sm.fit_resample(X_train,y_train)
X_train_SMOTE.shape

(144840, 656)

In [14]:
# LR again but with different variable name to keep the difference
LR = LogisticRegression(random_state=7, solver='saga',
                  multi_class='multinomial', max_iter=1000)
LR.fit(X_train_SMOTE, y_train_SMOTE)
pred = LR.predict(X_test)

print("accuracy: ", LR.score(X_test, y_test))
print("precision: ",precision_score(y_test,pred))
print("recall: ",recall_score(y_test,pred))
print("f1: ",f1_score(y_test,pred))

accuracy:  0.6156788764869255
precision:  0.06616052060737528
recall:  0.5224839400428265
f1:  0.1174488567990373


In [15]:
# Oh no! accuracy and precision has dropped significantly. But recall as increased by a lot, resulting in f1 score increase as well

# Lab | Random Forests

For this lab, you will be using the .CSV files provided in the `files_for_lab` folder.  These are cleaned versions of the learningSet data from the Case Study 'Healthcare for All'.   
You may continue in the Jupyter Notebook you created yesterday.  There is no need to fork and clone this Repo.

### Instructions

- Apply the Random Forests algorithm AFTER upscaling the data to deal with the imbalance.
- Use Feature Selections that you have learned in class to decide if you want to use all of the features (Variance Threshold, RFE, PCA, etc.)
- Re-run the Random Forest algorithm to determine if the Feature Selection has improved the results.
- Discuss the output and its impact in the business scenario. Is the cost of a false positive equals to the cost of the false negative? How would you change your algorithm or data in order to maximize the return of the business?

In [20]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(max_depth=20, # max number of questions to ask
                             min_samples_split=20, # amount of rows still considered at every question
                             min_samples_leaf =20, # ultimate answer based on at least this many rows
                             max_samples=0.8, # fraction of X-train to use in each tree
                             random_state=7)
clf.fit(X_train_SMOTE, y_train_SMOTE)
print(clf.score(X_train_SMOTE, y_train_SMOTE))
print(clf.score(X_test, y_test))

0.9743233913283623
0.9503222763716397


In [23]:
# accuracy is not bad, much better than the Logistic Regression model
pred = clf.predict(X_test)

print("precision: ",precision_score(y_test,pred))
print("recall: ",recall_score(y_test,pred))
print("f1: ",f1_score(y_test,pred))

precision:  0.1111111111111111
recall:  0.0021413276231263384
f1:  0.004201680672268908


In [28]:
# None of the above three metrics are better than the Logistic Regression model
# Let's use PCA to build/reduce features
from sklearn.decomposition import PCA
pca = PCA(0.80).fit(X_train_SMOTE)
pca.explained_variance_ratio_ # OK this looks cool, acceptable number of features adding up to explained variance ratio of 0.90
                              # But even the one with highest evr is not too high (0.09), I'll try anyway

array([0.09689771, 0.04342327, 0.03896155])

In [27]:
X_train_SMOTE_pca = pca.transform(X_train_SMOTE)

clf2 = RandomForestClassifier(max_depth=10, # max number of questions to ask
                             min_samples_split=20, # amount of rows still considered at every question
                             min_samples_leaf =20, # ultimate answer based on at least this many rows
                             random_state=7)
clf2.fit(X_train_SMOTE_pca, y_train_SMOTE)

pred2 = clf2.predict(pca.transform(X_test))

print("accuracy: ",clf2.score(pca.transform(X_test), y_test))
print("precision: ",precision_score(y_test,pred2))
print("recall: ",recall_score(y_test,pred2))
print("f1: ",f1_score(y_test,pred2))

accuracy:  0.7328512288424252
precision:  0.058524173027989825
recall:  0.2955032119914347
f1:  0.09769911504424779


In [29]:
# this forest has way better recall and f1 score than the previous before pca
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,pred2)

array([[13709,  4440],
       [  658,   276]], dtype=int64)

In [ ]:
# I would just use the logistic regression model.